In [1]:
!pip install -q mamba-ssm
!pip install -q causal-conv1d

In [2]:
import torch
import time
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

In [3]:
# config
torch.manual_seed(11)
device = "cuda"

In [4]:
with open("../input/for-use/shakespeare.txt", "r") as f:
    data = f.read()
print(data[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [5]:
chars = sorted(list(set(data)))
vocab_size = len(chars)

input_size = 1024
batch_size = 4

c2i = {c: i for i, c in enumerate(chars)}
i2c = {i: c for i, c in enumerate(chars)}

encode = lambda s: [c2i[c] for c in s]
decode = lambda l: "".join([i2c[i] for i in l])

data_ = torch.tensor(encode(data), dtype=torch.long)
n = int(len(data_) * 0.9)

train_data = data_[:n]
val_data = data_[n:]

In [6]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - input_size, (batch_size,))
    x = torch.stack([data[i : i + input_size] for i in ix])
    y = torch.stack([data[i + 1 : i + input_size + 1] for i in ix])
    x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    return x, y

model = MambaLMHeadModel(d_model=768,n_layer=12,vocab_size=vocab_size,device=device)
n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
loss_fct = torch.nn.CrossEntropyLoss()

In [7]:
# training
t_start = time.time()
max_iters = 1000
print_interval = 100
for iter in range(max_iters):
    xb, yb = get_batch("train")
    logits = model(xb).logits
    B, T, C = logits.shape
    logits = logits.view(B * T, C)
    yb = yb.view(B * T)
    loss = loss_fct(logits, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if iter % print_interval == 0:
        print('Iteration {} loss: {}'.format(iter + 1, loss.item()))

print("\n")
print(f"Number of parameters: {n_params}")
print(f"Training time:        {(time.time() - t_start)/60:.2f} min")

TypeError: causal_conv1d_fwd(): incompatible function arguments. The following argument types are supported:
    1. (arg0: torch.Tensor, arg1: torch.Tensor, arg2: Optional[torch.Tensor], arg3: Optional[torch.Tensor], arg4: bool) -> torch.Tensor

Invoked with: tensor([[[-0.4288, -0.7415,  0.2338,  ...,  0.4696, -0.7415,  0.1994],
         [-0.2644,  0.9622, -0.7267,  ..., -0.2105,  0.9622, -0.5146],
         [-1.2545, -0.0087,  0.3420,  ...,  0.3833, -0.0087,  0.5425],
         ...,
         [-0.0883, -0.8732,  0.4167,  ...,  0.7703, -0.8732,  0.5065],
         [-0.3103,  0.1674, -0.9188,  ..., -0.4987,  0.1674, -0.6746],
         [ 0.0817,  0.1187,  0.4305,  ...,  0.2771,  0.1187, -0.6790]],

        [[ 0.4696,  0.3384,  0.5967,  ...,  0.2312, -0.6446, -0.5635],
         [-0.2105, -0.9218,  0.6288,  ..., -0.8121,  0.4196, -0.1247],
         [ 0.3833, -0.1912, -0.0106,  ...,  0.5447,  0.8804, -0.5389],
         ...,
         [ 0.7703, -0.0258, -0.0229,  ...,  0.5308, -0.0322, -0.6238],


In [8]:
prompt_tokens = torch.tensor(encode("Shall I compare thee to a summer's "),dtype=torch.long, device=device).unsqueeze(1).T
res_tokens = model.generate(prompt_tokens, max_length=200,top_k=10,top_p=1.0,temperature=1.1,cg=True)
list_chars = res_tokens.tolist()[0]
print(decode(list_chars))

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Exception raised from c10_cuda_check_implementation at /usr/local/src/pytorch/c10/cuda/CUDAException.cpp:44 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >) + 0xae (0x7c12e295300e in /opt/conda/lib/python3.10/site-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&) + 0xf3 (0x7c12e29191cf in /opt/conda/lib/python3.10/site-packages/torch/lib/libc10.so)
frame #2: c10::cuda::c10_cuda_check_implementation(int, char const*, 